In [1]:
import pandas as pd

df = pd.read_csv("gym and diet recommendation1.csv")
print(df.isnull().sum())  # Check for missing values

ID              0
Sex             0
Age             0
Height          0
Weight          0
Hypertension    0
Diabetes        0
BMI             0
Level           0
Fitness Goal    0
Fitness Type    0
Exercises       0
Diet            0
dtype: int64


In [2]:
df['Calculated_BMI'] = df['Weight'] / (df['Height'] ** 2)

df['BMI_Discrepancy'] = abs(df['BMI'] - df['Calculated_BMI']) > 0.1
print(df[df['BMI_Discrepancy']][['ID', 'BMI', 'Calculated_BMI']])

df['BMI'] = df['Calculated_BMI']
df = df.drop(columns=['Calculated_BMI', 'BMI_Discrepancy'])

          ID   BMI  Calculated_BMI
14450  14451  70.0       24.677021


In [3]:
duplicates = df.duplicated(subset=df.columns.difference(['ID']), keep=False)
print(df[duplicates][['ID']])

df = df.drop_duplicates(subset=df.columns.difference(['ID']), keep='first')

          ID
0          1
1          2
2          3
3          4
4          5
...      ...
14575  14576
14576  14577
14577  14578
14578  14579
14579  14580

[4714 rows x 1 columns]


In [4]:
df['Hypertension'] = df['Hypertension'].str.capitalize()
df['Diabetes'] = df['Diabetes'].str.capitalize()
df['Level'] = df['Level'].str.capitalize()
df['Fitness Goal'] = df['Fitness Goal'].str.capitalize()
df['Fitness Type'] = df['Fitness Type'].str.capitalize()

print(df['Level'].value_counts())
print(df['Fitness Goal'].value_counts())
print(df['Fitness Type'].value_counts())

Level
Overweight     3362
Normal         3040
Underweight    2896
Obuse          2840
Name: count, dtype: int64
Fitness Goal
Weight loss    6202
Weight gain    5936
Name: count, dtype: int64
Fitness Type
Cardio fitness      6202
Muscular fitness    5936
Name: count, dtype: int64


In [5]:
inconsistent = df[
    ((df['Level'].isin(['Underweight', 'Normal'])) & (df['Fitness Goal'] == 'Weight Loss')) |
    ((df['Level'].isin(['Overweight', 'Obese'])) & (df['Fitness Goal'] == 'Weight Gain'))
]
print(inconsistent[['ID', 'Level', 'Fitness Goal']])

Empty DataFrame
Columns: [ID, Level, Fitness Goal]
Index: []


In [6]:
# Detect outliers using IQR
for col in ['Age', 'Height', 'Weight', 'BMI']:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    print(f"Outliers in {col}:\n", outliers[['ID', col]])

Outliers in Age:
 Empty DataFrame
Columns: [ID, Age]
Index: []
Outliers in Height:
           ID  Height
2017    2018    1.40
2018    2019    1.40
2019    2020    1.40
2020    2021    1.40
2021    2022    1.40
...      ...     ...
13626  13627    1.99
13627  13628    2.00
13628  13629    2.01
13629  13630    2.02
13630  13631    2.03

[69 rows x 2 columns]
Outliers in Weight:
 Empty DataFrame
Columns: [ID, Weight]
Index: []
Outliers in BMI:
           ID        BMI
2041    2042  50.000000
2042    2043  50.000000
2043    2044  50.000000
2044    2045  50.000000
2045    2046  50.000000
...      ...        ...
12824  12825  49.535132
12825  12826  49.535132
12826  12827  49.535132
13788  13789  46.484375
13789  13790  46.294510

[106 rows x 2 columns]


In [7]:
df['Sex'] = df['Sex'].map({'Male': 0, 'Female': 1})
df['Hypertension'] = df['Hypertension'].map({'No': 0, 'Yes': 1})
df['Diabetes'] = df['Diabetes'].map({'No': 0, 'Yes': 1})

print(df[['Sex', 'Hypertension', 'Diabetes']].value_counts())

Sex  Hypertension  Diabetes
0    0             0           2560
                   1           1579
     1             0           1578
                   1           1578
1    0             0           1211
                   1           1211
     1             0           1211
                   1           1210
Name: count, dtype: int64


In [8]:
df['Level'] = df['Level'].map({'Underweight': 0, 'Normal': 1, 'Overweight': 2, 'Obuse': 3})
print(df['Level'].value_counts())

Level
2    3362
1    3040
0    2896
3    2840
Name: count, dtype: int64


In [9]:
df['Fitness Goal'] = df['Fitness Goal'].map({'Weight gain': 0, 'Weight loss': 1})
df['Fitness Type'] = df['Fitness Type'].map({'Muscular fitness': 0, 'Cardio fitness': 1})
print(df['Fitness Goal'].value_counts())
print(df['Fitness Type'].value_counts())

Fitness Goal
1    6202
0    5936
Name: count, dtype: int64
Fitness Type
1    6202
0    5936
Name: count, dtype: int64


In [10]:
from sklearn.preprocessing import LabelEncoder
import uuid

le_exercises = LabelEncoder()
df['Exercises_Label'] = le_exercises.fit_transform(df['Exercises'])

translation_dict = {
    'Squats, deadlifts, bench presses, and overhead presses': 'اسکات،ددلیفت،پرس سینه،پرس بالای سر',
    'Squats, yoga, deadlifts, bench presses, and overhead presses': 'اسکات،یوگا،ددلیفت،پرس سینه،پرس بالای سر',
    'Brisk walking, cycling, swimming, running , or dancing.': 'پیاده‌روی سریع،دوچرخه‌سواری،شنا،دویدن یا رقص',
    'brisk walking, cycling, swimming, or dancing.': 'پیاده‌روی سریع،دوچرخه‌سواری،شنا یا رقص',
    'Walking, Yoga, Swimming.': 'پیاده‌روی،یوگا،شنا'
    
}

mapping_df = pd.DataFrame({
    'Exercise_Label': le_exercises.transform(le_exercises.classes_),
    'Exercise_String': le_exercises.classes_,
    'Exercise_String_Persian': [translation_dict.get(ex, 'ترجمه نشده') for ex in le_exercises.classes_]
})


mapping_df.to_csv('exercises_mapping.csv', index=False)
print("Encoded Exercises value counts:\n", df['Exercises_Label'].value_counts())
print("\nMapping preview:\n", mapping_df.head())

Encoded Exercises value counts:
 Exercises_Label
0    3423
1    3137
2    2799
3    1390
4    1389
Name: count, dtype: int64

Mapping preview:
    Exercise_Label                                    Exercise_String  \
0               0  Brisk walking, cycling, swimming, running , or...   
1               1  Squats, deadlifts, bench presses, and overhead...   
2               2  Squats, yoga, deadlifts, bench presses, and ov...   
3               3                           Walking, Yoga, Swimming.   
4               4      brisk walking, cycling, swimming, or dancing.   

                        Exercise_String_Persian  
0  پیاده‌روی سریع،دوچرخه‌سواری،شنا،دویدن یا رقص  
1            اسکات،ددلیفت،پرس سینه،پرس بالای سر  
2       اسکات،یوگا،ددلیفت،پرس سینه،پرس بالای سر  
3                            پیاده‌روی،یوگا،شنا  
4        پیاده‌روی سریع،دوچرخه‌سواری،شنا یا رقص  


In [11]:
print("Pure Diet value counts:\n", df['Diet'].value_counts())

Pure Diet value counts:
 Diet
Vegetables: (Garlic, Mushroom, Green Papper, Icebetg Lettuce); Protein Intake: (Baru Nuts, Beech Nuts, Hemp Seeds, Cheese Spandwich); Juice: (Apple Juice, Mango juice,and Beetroot juice)                                                                      4155
Vegetables: (Broccoli, Carrots, Spinach, Lettuce, Onion); Protein Intake: (Cheese, Cattoge cheese, Skim Milk, Law fat Milk, and Baru Nuts); Juice: (Fruit Juice, Aloe vera juice, Cold-pressed juice, and Watermelon juice)                                     2025
Vegetables: (Garlic, Roma Tomatoes, Capers, Green Papper, and Iceberg Lettuce); Protein Intake: (Cheese Sandwich, Baru Nuts, Beech Nuts, Squash Seeds, Mixed Teff, peanut butter, and jelly sandwich); Juice: (Apple juice, beetroot juice, and mango juice)    1397
Vegetables: (Mixed greens, cherry tomatoes, cucumbers, bell peppers, carrots, celery, bell peppers);Protein Intake: (Chicken, fish, tofu, or legumes); Juice : (Green juice,kale, spinach, 

In [12]:
def impute_diet(row):
    if row['Diet'].strip().lower() == 'diet':
        if row['Level'] in ['Underweight', 'Normal']:
            return 'Vegetables: (Carrots, Sweet Potato, and Lettuce); Protein Intake: (Red meats, poultry, fish, eggs, dairy products, legumes, and nuts); Juice: (Fruit juice, watermelon juice, carrot juice, apple juice and mango juice)'
        elif row['Level'] in ['Overweight', 'Obese']:
            return 'Vegetables: (Mixed greens, cherry tomatoes, cucumbers, bell peppers, carrots, celery, bell peppers); Protein Intake: (Chicken, fish, tofu, or legumes); Juice: (Green juice, kale, spinach, cucumber, celery, and apple)'
        else:
            return 'Vegetables: (Garlic, Mushroom, Green Papper, Icebetg Lettuce); Protein Intake: (Baru Nuts, Beech Nuts, Hemp Seeds, Cheese Spandwich); Juice: (Apple Juice, Mango juice, and Beetroot juice)'
    return row['Diet']

df['Diet'] = df.apply(impute_diet, axis=1)

le_diet = LabelEncoder()
df['Diet_Label'] = le_diet.fit_transform(df['Diet'])

translation_dict = {
    'Vegetables: (Broccoli, Carrots, Spinach, Lettuce, Onion); Protein Intake: (Cheese, Cattoge cheese, Skim Milk, Law fat Milk, and Baru Nuts); Juice: (Fruit Juice, Aloe vera juice, Cold-pressed juice, and Watermelon juice)': 
        'سبزیجات: (بروکلی، هویج، اسفناج، کاهو، پیاز); پروتئین: (پنیر، پنیر کاتیج، شیر بدون چربی، شیر کم‌چرب، آجیل بارو); آبمیوه: (آبمیوه، آب آلوئه‌ورا، آبمیوه سردفشرده، آب هندوانه)',
    'Vegetables: (Carrots, Sweet Potato, Lettuce); Protein Intake: (Red meats, poultry, fish, eggs, dairy products, legumes, and nuts); Juice: (Fruit juice, watermelon juice, carrot juice, apple juice and mango juice)': 
        'سبزیجات: (هویج، سیب‌زمینی شیرین، کاهو); پروتئین: (گوشت قرمز، ماکیان، ماهی، تخم‌مرغ، محصولات لبنی، حبوبات، آجیل); آبمیوه: (آبمیوه، آب هندوانه، آب هویج، آب سیب، آب انبه)',
    'Vegetables: (Carrots, Sweet Potato, and Lettuce); Protein Intake: (Red meats, poultry, fish, eggs, dairy products, legumes, and nuts); Juice: (Fruit juice, watermelon juice, carrot juice, apple juice and mango juice)': 
        'سبزیجات: (هویج، سیب‌زمینی شیرین، کاهو); پروتئین: (گوشت قرمز، ماکیان، ماهی، تخم‌مرغ، محصولات لبنی، حبوبات، آجیل); آبمیوه: (آبمیوه، آب هندوانه، آب هویج، آب سیب، آب انبه)',
    'Vegetables: (Garlic, Mushroom, Green Papper, Icebetg Lettuce); Protein Intake: (Baru Nuts, Beech Nuts, Hemp Seeds, Cheese Spandwich); Juice: (Apple Juice, Mango juice, and Beetroot juice)': 
        'سبزیجات: (سیر، قارچ، فلفل سبز، کاهوی آیسبرگ); پروتئین: (آجیل بارو، آجیل راش، دانه کنف، ساندویچ پنیر); آبمیوه: (آب سیب، آب انبه، آب چغندر)',
    'Vegetables: (Garlic, Mushroom, Green Papper, Icebetg Lettuce); Protein Intake: (Baru Nuts, Beech Nuts, Hemp Seeds, Cheese Spandwich); Juice: (Apple Juice, Mango juice,and Beetroot juice)': 
        'سبزیجات: (سیر، قارچ، فلفل سبز، کاهوی آیسبرگ); پروتئین: (آجیل بارو، آجیل راش، دانه کنف، ساندویچ پنیر); آبمیوه: (آب سیب، آب انبه، آب چغندر)',
    'Vegetables: (Garlic, Roma Tomatoes, Capers and Iceberg Lettuce); Protein Intake: (Cheese Standwish, Baru Nuts, Beech Nuts, Squash Seeds, and Mixed Teff); Juice: (Apple juice, beetroot juice and mango juice)': 
        'سبزیجات: (سیر، گوجه‌فرنگی روما، کبر، کاهوی آیسبرگ); پروتئین: (ساندویچ پنیر، آجیل بارو، آجیل راش، دانه کدو، تف مخلوط); آبمیوه: (آب سیب، آب چغندر، آب انبه)',
    'Vegetables: (Garlic, Roma Tomatoes, Capers, Green Papper, and Iceberg Lettuce); Protein Intake: (Cheese Sandwich, Baru Nuts, Beech Nuts, Squash Seeds, Mixed Teff, peanut butter, and jelly sandwich); Juice: (Apple juice, beetroot juice, and mango juice)': 
        'سبزیجات: (سیر، گوجه‌فرنگی روما، کبر، فلفل سبز، کاهوی آیسبرگ); پروتئین: (ساندویچ پنیر، آجیل بارو، آجیل راش، دانه کدو، تف مخلوط، کره بادام‌زمینی، ساندویچ ژله); آبمیوه: (آب سیب، آب چغندر، آب انبه)',
    'Vegetables: (Garlic, mushroon, green papper and water chestnut);Protein Intake: ( Baru Nuts, Beech Nuts, and black walnut); Juice : (Apple juice, Mango, and Beetroot Juice)': 
        'سبزیجات: (سیر، قارچ، فلفل سبز، شاه‌بلوط آبی); پروتئین: (آجیل بارو، آجیل راش، گردوی سیاه); آبمیوه: (آب سیب، آب انبه، آب چغندر)',
    'Vegetables: (Garlic, mushroon, green papper);Protein Intake: ( Baru Nuts, Beech Nuts, and Hemp Seeds); Juice : (Apple juice, Mango, and Beetroot Juice)': 
        'سبزیجات: (سیر، قارچ، فلفل سبز); پروتئین: (آجیل بارو، آجیل راش، دانه کنف); آبمیوه: (آب سیب، آب انبه، آب چغندر)',
    'Vegetables: (Mixed greens, cherry tomatoes, cucumbers, bell peppers, carrots, celery, bell peppers);Protein Intake: (Chicken, fish, tofu, or legumes); Juice : (Green juice,kale, spinach, cucumber, celery, and apple)': 
        'سبزیجات: (سبزیجات مخلوط، گوجه‌فرنگی گیلاسی، خیار، فلفل دلمه‌ای، هویج، کرفس، فلفل دلمه‌ای); پروتئین: (مرغ، ماهی، توفو، حبوبات); آبمیوه: (آب سبز، کلم کیل، اسفناج، خیار، کرفس، سیب)',
    'Vegetables: (Tomatoes, Garlic, leafy greens, broccoli, carrots, and bell peppers); Protein Intake: (poultry, fish, tofu, legumes, and low-fat dairy products); Juice: (Apple juice, beetroot juice and mango juice)': 
        'سبزیجات: (گوجه‌فرنگی، سیر، سبزیجات برگی، بروکلی، هویج، فلفل دلمه‌ای); پروتئین: (ماکیان، ماهی، توفو، حبوبات، محصولات لبنی کم‌چرب); آبمیوه: (آب سیب، آب چغندر، آب انبه)'
}

mapping_df = pd.DataFrame({
    'Diet_Label': le_diet.transform(le_diet.classes_),
    'Diet_String_English': le_diet.classes_,
    'Diet_String_Persian': [translation_dict.get(diet, 'ترجمه نشده') for diet in le_diet.classes_]
})

mapping_df.to_csv('diet_mapping.csv', index=False, encoding='utf-8')

print("Number of unique diets:", df['Diet'].nunique())
print("\nEncoded Diet value counts:\n", df['Diet_Label'].value_counts())
print("\nMapping preview:\n", mapping_df)


Number of unique diets: 11

Encoded Diet value counts:
 Diet_Label
4     4155
0     2025
6     1397
9      939
5      703
10     702
7      696
8      696
2      399
1      399
3       27
Name: count, dtype: int64

Mapping preview:
     Diet_Label                                Diet_String_English  \
0            0  Vegetables: (Broccoli, Carrots, Spinach, Lettu...   
1            1  Vegetables: (Carrots, Sweet Potato, Lettuce); ...   
2            2  Vegetables: (Carrots, Sweet Potato, and Lettuc...   
3            3  Vegetables: (Garlic, Mushroom, Green Papper, I...   
4            4  Vegetables: (Garlic, Mushroom, Green Papper, I...   
5            5  Vegetables: (Garlic, Roma Tomatoes, Capers and...   
6            6  Vegetables: (Garlic, Roma Tomatoes, Capers, Gr...   
7            7  Vegetables: (Garlic, mushroon, green papper an...   
8            8  Vegetables: (Garlic, mushroon, green papper);P...   
9            9  Vegetables: (Mixed greens, cherry tomatoes, cu...   
10      

In [13]:
df.to_csv('GymAndDietRecommendationCleaned.csv', index=False, encoding='utf-8')